<a href="https://colab.research.google.com/github/NSuprotivniy/2017-big-data/blob/master/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import model_from_json
from glob import glob
import numpy as np
from os.path import join, exists
from os import mkdir
from google.colab import drive
from math import ceil

Using TensorFlow backend.


In [2]:
drive.mount("/content/nsuprotivniy/")
model_path = "/content/nsuprotivniy/My Drive/Colab Notebooks/data/LCC_FASD/LCC_FASD_training/model"
data_path = "/content/nsuprotivniy/My Drive/Colab Notebooks/data/LCC_FASD/LCC_FASD_evaluation/cropped"

Drive already mounted at /content/nsuprotivniy/; to attempt to forcibly remount, call drive.mount("/content/nsuprotivniy/", force_remount=True).


In [0]:
output_path = "/content/nsuprotivniy/My Drive/Colab Notebooks/data/LCC_FASD/LCC_FASD_evaluation/result"
if (not exists(output_path)):
  mkdir(output_path)

In [0]:
class Generator():
  def __init__(self, data_path, batch_size=8):
    files = glob(join(data_path, "*.npy"))
    self.batches = np.array_split(files, ceil(len(files) / batch_size))
    
  def len(self):
    return len(self.batches)
  
  def get_batch(self, idx):
    batch = self.batches[idx]
    X = np.array([np.load(path) for path in batch])
    file_names = [path.split("/")[-1].split(".npy")[0] for path in batch]
    return X, file_names
      

In [0]:
gen = Generator(data_path)

In [6]:
with open(join(model_path, 'model.json'), 'r') as f:
  loaded_model_json = f.read()
model = model_from_json(loaded_model_json)
model.load_weights(join(model_path, "model.h5"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
for i in range(gen.len()):
  X, file_names = gen.get_batch(i)
  predicted = model.predict(X, verbose=1)
  with open(join(output_path, "prediction_result.txt"), "a") as f:
    for file, proba in zip(file_names, predicted):
      f.write("{}; {}\n".format(file, proba))

7/7 [==============================] - 0s 7ms/step
